In [17]:
import numpy as np
import pandas as pd

import pandas_datareader.data as web
from pandas_datareader import wb

import dask
import dask.dataframe as dd

import datetime

In [18]:
def worldBandData(indicator, endDate, col_name):
    df = wb.download(country='All', indicator=indicator, start=1960, end=endDate)
    df.columns = ['GDP_growth_annual_perc']
    return df

In [22]:
def loadDataSouces():
    df = pd.read_csv('../data/raw/dataSource.csv')
    return df

In [ ]:
def saveData(df, source, indicator):
    df.to_parquet('../data/raw/{0}_{1}.parquet'.format(source, indicator))

In [31]:
dataSources = loadDataSouces()

In [32]:
dataSources

,connector,source,Indicator,col_name
0,pandas_datareader,WorldBank,NY.GDP.MKTP.KD.ZG,GDP_growth_annual_perc
1,pandas_datareader,WorldBank,SP.POP.GROW,NaN


In [26]:
wb.download(country='All', indicator="SP.POP.GROW", start=1960, end=2020)

SP.POP.GROW
country    year             
Arab World 2020          NaN
           2019     1.924693
           2018     1.915913
           2017     1.949024
           2016     2.019087
...                      ...
Zimbabwe   1964     3.391104
           1963     3.395952
           1962     3.378041
           1961     3.342091
           1960          NaN

[16104 rows x 1 columns]

In [7]:
GDP_growth_annual_perc = wb.download(country='All', indicator="NY.GDP.MKTP.KD.ZG", start=1960, end=2020)

In [10]:
GDP_growth_annual_perc.columns = ['GDP_growth_annual_perc']

In [16]:
GDP_growth_annual_perc.xs('United Kingdom', level=0)

,GDP_growth_annual_perc
year,
2020,NaN
2019,1.408642
2018,1.340721
2017,1.892077
2016,1.918155
...,...
1964,5.533659
1963,4.874384
1962,1.102910


In [28]:
GDP_growth_annual_perc.to_parquet('../data/raw/WorldBank_GDP_growth_annual_perc.parquet')

In [11]:
f = web.DataReader('', 'stooq')

In [12]:
f

,Open,High,Low,Close,Volume
Date,,,,,
2020-10-09,87.410,88.7000,87.350,87.700,6005779
2020-10-08,87.270,88.5800,86.920,87.340,5535131
2020-10-07,86.160,87.2977,86.030,87.070,5899660
2020-10-06,87.970,88.0600,85.710,85.890,8743761
2020-10-05,86.490,88.1800,86.210,87.930,5984220
...,...,...,...,...,...
2015-10-16,45.749,46.3000,45.343,45.943,9034687
2015-10-15,44.148,45.6760,43.767,45.626,9574590
2015-10-14,43.962,44.3930,43.533,43.838,10738195


Int64Index([ 9,  8,  7,  6,  5,  2,  1, 30, 29, 28,
            ...
            23, 22, 21, 20, 19, 16, 15, 14, 13, 12],
           dtype='int64', name='Date', length=1269)